# Initialize our Spark Session & Import Pyspark Functions

In [0]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("goal conversion metric").getOrCreate()

spark.sparkContext.setLogLevel("WARN")

# Which striker is the most efficient or clincial in front of goal

## Read Data Sources


In [0]:
apperarances = spark.read.csv(
    path="dbfs:/FileStore/tables/appearances.csv",
    sep=",",
    header=True,
    inferSchema=True
)
season = spark.read.csv(
    path="dbfs:/FileStore/tables/games.csv",
    sep=",",
    header=True,
    inferSchema=True
)
leagues = spark.read.csv(
    path="dbfs:/FileStore/tables/leagues.csv",
    sep=",",
    header=True,
    inferSchema=True
)
players = spark.read.csv(
    "dbfs:/FileStore/tables/players.csv",
    sep=",",
     header=True,
     inferSchema=True
)

players.printSchema()
leagues.printSchema()
season.printSchema()
apperarances.printSchema()

root
 |-- gameID: integer (nullable = true)
 |-- playerID: integer (nullable = true)
 |-- goals: integer (nullable = true)
 |-- ownGoals: integer (nullable = true)
 |-- shots: integer (nullable = true)
 |-- xGoals: double (nullable = true)
 |-- xGoalsChain: double (nullable = true)
 |-- xGoalsBuildup: double (nullable = true)
 |-- assists: integer (nullable = true)
 |-- keyPasses: integer (nullable = true)
 |-- xAssists: double (nullable = true)
 |-- position: string (nullable = true)
 |-- positionOrder: integer (nullable = true)
 |-- yellowCard: integer (nullable = true)
 |-- redCard: integer (nullable = true)
 |-- time: integer (nullable = true)
 |-- substituteIn: integer (nullable = true)
 |-- substituteOut: integer (nullable = true)
 |-- leagueID: integer (nullable = true)



## Data Filtering and Transformation

In [0]:
distinct_positions = apperarances.select("position").dropDuplicates()
positions = ['FW', 'FWR', 'FWL']
apperarances = apperarances.filter(F.col("position").isin(positions))
apperarances = apperarances.drop( *[x for x in apperarances.columns if x not in ['gameID', 'playerID', 'goals', 'shots', 'leagueID']])
season = season.drop( *[x for x in season.columns if x not in ['gameID','season',]])
leagues = leagues.drop('understatNotation')
leagues = leagues.withColumnRenamed('name', 'league')
players = players.withColumnRenamed('name','player_name')

+--------+
|position|
+--------+
|     DMC|
|      DR|
|      DC|
|      FW|
|      DL|
|     FWR|
|     DML|
|      GK|
|      MR|
|     Sub|
|     AMC|
|     DMR|
|      MC|
|     AML|
|      ML|
|     AMR|
|     FWL|
+--------+




## Join Operations and Aggregation

In [0]:
apperarances = apperarances.join(
    other=season, on="gameID", how="left"
)
apperarances = apperarances.groupBy("playerID", 'leagueID', 'season').agg(F.sum("goals").alias("total_goals"),F.sum("shots").alias("total_shots"))

+------+--------+-----+-----+--------+------+
|gameID|playerID|goals|shots|leagueID|season|
+------+--------+-----+-----+--------+------+
|    81|     629|    0|    2|       1|  2015|
|    81|     647|    0|    3|       1|  2015|
|    82|     465|    0|    2|       1|  2015|
|    82|     468|    0|    2|       1|  2015|
|    82|     672|    0|    0|       1|  2015|
|    82|     890|    0|    2|       1|  2015|
|    82|     679|    0|    0|       1|  2015|
|    83|     594|    0|    2|       1|  2015|
|    83|     574|    0|    3|       1|  2015|
|    84|     755|    1|    5|       1|  2015|
|    84|     754|    0|    2|       1|  2015|
|    84|     735|    1|    3|       1|  2015|
|    85|     467|    0|    2|       1|  2015|
|    85|     882|    0|    1|       1|  2015|
|    86|     802|    0|    1|       1|  2015|
|    86|     717|    1|    5|       1|  2015|
|    87|     850|    1|    3|       1|  2015|
|    87|     841|    1|    2|       1|  2015|
|    88|     502|    0|    5|     

## Further Filtering

In [0]:
minimum_shot_threshold = 20
season = 2019
apperarances = apperarances.filter(
    (F.col('total_shots')>= minimum_shot_threshold) & 
    (F.col('season') == season)
    )

## Final Data Preparation and Analysis

In [0]:
full_shooting_data = apperarances.join(
    other=leagues, on='leagueID', how="left"
    ).join(
        other=players, on="playerID", how="left"
    )

+--------+--------+------+-----------+-----------+--------------+------------------+
|playerID|leagueID|season|total_goals|total_shots|league        |player_name       |
+--------+--------+------+-----------+-----------+--------------+------------------+
|2566    |4       |2019  |9          |50         |La Liga       |Roger             |
|8140    |3       |2019  |4          |20         |Bundesliga    |Munas Dabbur      |
|839     |1       |2019  |2          |28         |Premier League|Shane Long        |
|1592    |2       |2019  |11         |51         |Serie A       |Jo�o Pedro        |
|3243    |5       |2019  |4          |46         |Ligue 1       |Ga�tan Charbonnier|
+--------+--------+------+-----------+-----------+--------------+------------------+
only showing top 5 rows



## Analysis of highest goal conversion rates 2019

In [0]:
all_shots = full_shooting_data.withColumn(
    "goal_conversion",
    (F.col("total_goals") / F.col("total_shots"))
).orderBy(F.col("goal_conversion").desc()).show()

+--------+--------+------+-----------+-----------+--------------+------------------+-------------------+
|playerID|leagueID|season|total_goals|total_shots|        league|       player_name|    goal_conversion|
+--------+--------+------+-----------+-----------+--------------+------------------+-------------------+
|    5220|       3|  2019|          8|         20|    Bundesliga|       Kai Havertz|                0.4|
|    1513|       5|  2019|         10|         30|       Ligue 1|      Mauro Icardi| 0.3333333333333333|
|    7142|       5|  2019|         16|         52|       Ligue 1|    Moussa Dembele| 0.3076923076923077|
|    2400|       4|  2019|         11|         37|       La Liga|       Ra�l Garc�a| 0.2972972972972973|
|    8035|       2|  2019|          7|         24|       Serie A| Samuel Di Carmine| 0.2916666666666667|
|    8260|       3|  2019|          7|         24|    Bundesliga|    Erling Haaland| 0.2916666666666667|
|    6980|       2|  2019|         20|         70|     